In [1]:
# The code was removed by Watson Studio for sharing.

### Import all required libraries - numpy, pandas, json, requests, etc.

In [113]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim


import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

from scipy import stats

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Scraping the wiki page using BeautifulSoup - import BeautifulSoup and install lxml

In [4]:
from bs4 import BeautifulSoup
!conda install -y lxml

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libgcc-ng: 7.2.0-h7cc24e2_2     --> 8.2.0-hdf63c60_1    
    libxml2:   2.9.4-h6b072ca_5     --> 2.9.8-hf84eae3_0    
    libxslt:   1.1.29-hcf9102b_5    --> 1.1.32-h1312cb7_0   
    lxml:      4.1.0-py35ha401a81_0 --> 4.2.5-py35hefd8a0e_0

libgcc-ng-8.2. 100% |################################| Time: 0:00:00  22.32 MB/s
libxml2-2.9.8- 100% |################################| Time: 0:00:00  70.64 MB/s
libxslt-1.1.32 100% |################################| Time: 0:00:00  58.61 MB/s
lxml-4.2.5-py3 100% |################################| Time: 0:00:00  68.81 MB/s


### Get the url and print after scraping using BeautifulSoup

In [5]:
webURL = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(webURL,'lxml')
#print(soup.prettify()) # tested and then commented only to not show entire o/p to maintain readability

### Find the table wikitable sortable

In [6]:
postCodeTable = soup.find('table',{'class':'wikitable sortable'})
#postCodeTable

In [7]:
postalCode = []
borough = []
neighborhood = []

for row in postCodeTable.find_all('tr')[1:]:  # skipping first row for header
    boroughCell = row.find_all('td')[1] # processing boroughs - second coloumn first to skip non assigned ones 
    if (boroughCell.text.strip() != 'Not assigned'):
        borough.append(boroughCell.text.strip())
        postCell = row.find_all('td')[0]  #  first coloumn
        postalCode.append(postCell.text.strip())
        neighborhoodCell = row.find_all('td')[2] #third coloumn
        if (neighborhoodCell.text.strip() == 'Not assigned'):
            neighborhood.append(boroughCell.text.strip())
        else:
            neighborhood.append(neighborhoodCell.text.strip())

### Load into a pandas df and using groupby and join get the final required output

In [125]:
df = pd.DataFrame()
df['PostalCode'] = postalCode
df['Borough'] = borough
df['Neighborhood'] = neighborhood
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### df.shape to print number of rows of df including header

In [9]:
df.shape[0]

212

In [10]:
!conda install -c conda-forge geocoder --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geocoder:   1.38.1-py_0  conda-forge
    orderedset: 2.0-py35_0   conda-forge
    ratelim:    0.1.6-py35_0 conda-forge

orderedset-2.0 100% |################################| Time: 0:00:00   3.68 MB/s
ratelim-0.1.6- 100% |################################| Time: 0:00:00   7.66 MB/s
geocoder-1.38. 100% |################################| Time: 0:00:00  34.66 MB/s


In [11]:
# geocoder latlong does not seem to work always, using the csv option instead

geospat_df = pd.read_csv('http://cocl.us/Geospatial_data')
#geospat_df.head()

In [12]:
df_toronto = df.set_index('PostalCode').join(geospat_df.set_index('Postal Code'), how = 'inner').reset_index()
df_toronto

,index,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Rouge,43.806686,-79.194353
1,M1B,Scarborough,Malvern,43.806686,-79.194353
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1C,Scarborough,Rouge Hill,43.784535,-79.160497
4,M1C,Scarborough,Port Union,43.784535,-79.160497
5,M1E,Scarborough,Guildwood,43.763573,-79.188711
6,M1E,Scarborough,Morningside,43.763573,-79.188711
7,M1E,Scarborough,West Hill,43.763573,-79.188711
8,M1G,Scarborough,Woburn,43.770992,-79.216917
9,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
df_toronto.dtypes.index
#df_toronto.set_index('Neighborhood')

Index(['index', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

In [14]:
#Read demographic data
path = 'https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_neighbourhood_profiles.csv'
df_toronto_demographics = pd.read_csv(path, encoding = "ISO-8859-1", engine='python')
df_toronto_demographics.head(5)

,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No Designat

In [15]:
#Lets take the section which details of each neighborhood on the SouthAsian origin. Also we will include the total population row.
df_toronto_demo = df_toronto_demographics.iloc[np.r_[2,1579],:].copy()
#df_toronto_demo

In [16]:
df_toronto_demo.drop(df_toronto_demo.columns[[0,1,2,3,4]],axis=1,inplace=True)

In [17]:
df_toronto_demo.dtypes.index

Index(['Agincourt North', 'Agincourt South-Malvern West', 'Alderwood', 'Annex',
       'Banbury-Don Mills', 'Bathurst Manor', 'Bay Street Corridor',
       'Bayview Village', 'Bayview Woods-Steeles', 'Bedford Park-Nortown',
       ...
       'Willowdale West', 'Willowridge-Martingrove-Richview', 'Woburn',
       'Woodbine Corridor', 'Woodbine-Lumsden', 'Wychwood', 'Yonge-Eglinton',
       'Yonge-St.Clair', 'York University Heights', 'Yorkdale-Glen Park'],
      dtype='object', length=140)

In [18]:
#df_toronto_demo.set_index('Characteristic',inplace=True)

In [19]:
df_T = df_toronto_demo.T
#df_T

In [20]:
df_T.dtypes.index

Int64Index([2, 1579], dtype='int64')

In [21]:
df_T.reset_index()

,index,2,1579
0,Agincourt North,"29,113","5,260"
1,Agincourt South-Malvern West,"23,757","4,170"
2,Alderwood,"12,054",575
3,Annex,"30,526","1,320"
4,Banbury-Don Mills,"27,695","2,940"
5,Bathurst Manor,"15,873",565
6,Bay Street Corridor,"25,797","2,515"
7,Bayview Village,"21,396","1,410"
8,Bayview Woods-Steeles,"13,154",805
9,Bedford Park-Nortown,"23,236",535


In [22]:
df_T.columns=['Total_population','South_Asian_Population']

In [23]:
df_T.dtypes

Total_population          object
South_Asian_Population    object
dtype: object

In [24]:
#Merge this table with the post code and location tables
df_neighborhood = pd.merge(df_toronto, df_T, left_on='Neighborhood',right_index=True)

In [126]:
df_neighborhood

,index,Borough,Neighborhood,Latitude,Longitude,Total_population,South_Asian_Population,South_Asian_Percentage
70,M4H,East York,Thorncliffe Park,43.705369,-79.349372,21108.0,9545.0,45.219822
0,M1B,Scarborough,Rouge,43.806686,-79.194353,46496.0,20265.0,43.584394
1,M1B,Scarborough,Malvern,43.806686,-79.194353,43794.0,18085.0,41.295611
8,M1G,Scarborough,Woburn,43.770992,-79.216917,53485.0,21505.0,40.207535
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497,12494.0,4685.0,37.497999
16,M1L,Scarborough,Oakridge,43.711112,-79.284577,13845.0,4750.0,34.308415
52,M3C,North York,Flemingdon Park,43.725900,-79.340923,21933.0,7460.0,34.012675
10,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,16724.0,5535.0,33.096149
6,M1E,Scarborough,Morningside,43.763573,-79.188711,17455.0,5370.0,30.764824
22,M1P,Scarborough,Dorset Park,43.757410,-79.273304,25003.0,7380.0,29.516458


### To normalize this lets take % of South Asians to the population in the neighborhood

In [26]:
df_neighborhood.dtypes

index                      object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Total_population           object
South_Asian_Population     object
dtype: object

In [27]:
df_neighborhood.Total_population = df_neighborhood.Total_population.apply(lambda x: float(x.split()[0].replace(',', '')))
df_neighborhood.Total_population=df_neighborhood.Total_population.astype(float)
df_neighborhood.South_Asian_Population = df_neighborhood.South_Asian_Population.apply(lambda y: float(y.split()[0].replace(',', '')))
df_neighborhood.South_Asian_Population=df_neighborhood.South_Asian_Population.astype(float)

In [28]:
df_neighborhood.dtypes

index                      object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Total_population          float64
South_Asian_Population    float64
dtype: object

In [29]:
df_neighborhood['South_Asian_Percentage'] = (df_neighborhood['South_Asian_Population'] / df_neighborhood['Total_population'])*100

In [30]:
df_neighborhood.sort_values(['South_Asian_Percentage'], ascending=False, inplace=True)

In [31]:
df_neighborhood.reset_index(drop=True)

,index,Borough,Neighborhood,Latitude,Longitude,Total_population,South_Asian_Population,South_Asian_Percentage
0,M4H,East York,Thorncliffe Park,43.705369,-79.349372,21108.0,9545.0,45.219822
1,M1B,Scarborough,Rouge,43.806686,-79.194353,46496.0,20265.0,43.584394
2,M1B,Scarborough,Malvern,43.806686,-79.194353,43794.0,18085.0,41.295611
3,M1G,Scarborough,Woburn,43.770992,-79.216917,53485.0,21505.0,40.207535
4,M1C,Scarborough,Highland Creek,43.784535,-79.160497,12494.0,4685.0,37.497999
5,M1L,Scarborough,Oakridge,43.711112,-79.284577,13845.0,4750.0,34.308415
6,M3C,North York,Flemingdon Park,43.725900,-79.340923,21933.0,7460.0,34.012675
7,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,16724.0,5535.0,33.096149
8,M1E,Scarborough,Morningside,43.763573,-79.188711,17455.0,5370.0,30.764824
9,M1P,Scarborough,Dorset Park,43.757410,-79.273304,25003.0,7380.0,29.516458


In [ ]:
#Create a priority rating profile of neighborhood from the population parameter
dfPriorityNeighborhood = []
dfPriorityNeighborhood = pd.DataFrame(df_neighborhood, columns=['index','Borough','Neighborhood','Latitude','Longitude','Total_population','South_Asian_Population','South_Asian_Percentage','Priority'])
dfPriorityNeighborhood.drop(columns=['index','Borough','Latitude', 'Longitude', 'Total_population','South_Asian_Percentage','South_Asian_Population'], inplace=True)
dfPriorityNeighborhood.reset_index(drop=True, inplace=True)
#Assign a rating column for us to use it for recommendation
dfPriorityNeighborhood['Priority']= '5'
dfPriorityNeighborhood['Priority'][5:10]='4.5'
dfPriorityNeighborhood['Priority'][10:20]='4'
dfPriorityNeighborhood['Priority'][20:]='3.5'

In [109]:
dfPriorityNeighborhood

,Neighborhood,Priority
0,Thorncliffe Park,5
1,Rouge,5
2,Malvern,5
3,Woburn,5
4,Highland Creek,5
5,Oakridge,4.5
6,Flemingdon Park,4.5
7,Scarborough Village,4.5
8,Morningside,4.5
9,Dorset Park,4.5


### Lets map Toronto and neighborhoods now

In [33]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [34]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_neighborhood['Latitude'], df_neighborhood['Longitude'], df_neighborhood['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Now lets get the best neigborhood based on both the above population data and some of foursquare inputs

### Exploring using Foursquare API

In [35]:
# The code was removed by Watson Studio for sharing.

### Lets find the places where there are Asian/Indian restaurants, food, shops and services so that there is a possibility of tie-ups with them for the store to deliver ther fresh requirements. Additionally we will look for the reachability for footfalls in these venues using Metros or Busstops. 

### First let us write a function to get details of venues within 500m radius in each of the above neighborhood

In [36]:
# Lets look at the fourt categories of interest
categories = {
    'Indian Restaurant': '4bf58dd8d48988d10f941735',
    'Shop & Service': '4d4b7105d754a06378d81259',
    'Bus Stop': '52f2ab2ebcbc57f1066b8b4f',
    'Metro Station': '4bf58dd8d48988d1fd931735'
}

In [37]:
', '.join([ category for category in categories])

'Indian Restaurant, Metro Station, Bus Stop, Shop & Service'

In [38]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [39]:
print(categories['Indian Restaurant'])
for category, category_id in categories.items():
    print(category)

4bf58dd8d48988d10f941735
Indian Restaurant
Metro Station
Bus Stop
Shop & Service


In [40]:
def getNearbyVenueDetails(Neighborhood_fn, Latitude_fn, Longitude_fn, category):
    radius=500 # limiting the radius to 500
    limit=30 #limiting the venues to 30 
    venueList=[]

    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&query={}&ll={},{}&radius={}&limit={}&category={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                category,
                Latitude_fn, 
                Longitude_fn, 
                radius, 
                limit,
                categories[category])
                    
     
    # make the GET request
    results = requests.get(url).json()
        
    venues = results['response']['venues']
        
    # transform venues into a dataframe
    df = json_normalize(venues)
        
    cols = ['Venue','Venue Latitude','Venue Longitude']    
    if( len(df) == 0 ):        
        df = pd.DataFrame(columns=cols)
    else:        
        df = df[['name','location.lat','location.lng']]
        df.columns = cols
            
    return df

In [41]:
dfNearbyVenues=[]
dfNearbyVenues = pd.DataFrame(columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue Category',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude'])
dfFinalVenues=[]
dfFinalVenues = pd.DataFrame(columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue Category',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude'])
#counter = 0
#Loop through
for (index, Borough, Neighborhood, Latitude, Longitude, Total_population, South_Asian_Population, South_Asian_Percentage) in df_neighborhood.itertuples(index=False):
    ### For our analysis and avoiding exceeding Foursquare API calls for each neighborhood, we will concentrate on the top 15 Neighborhoods
 #   if (counter < 15):
        for category, category_id in categories.items():
            #Get the venue details
            dfNearbyVenues = getNearbyVenueDetails(Neighborhood, Latitude, Longitude, category)
            dfNearbyVenues['Neighborhood'] = Neighborhood
            dfNearbyVenues['Latitude'] = Latitude
            dfNearbyVenues['Longitude'] = Longitude
            dfNearbyVenues['Venue Category'] = category
            dfFinalVenues = dfFinalVenues.append(dfNearbyVenues)
   # counter = counter + 1
        
#Saving this as maxing out daily limits while testing on calls to Foursqaure with free account
project.save_data(data=dfFinalVenues.to_csv(index=False),file_name='venues12.csv',overwrite=True)
            

2 "Indian Restaurant" venues are found within 500 m of location
1 "Metro Station" venues are found within 500 m of location
5 "Bus Stop" venues are found within 500 m of location
1 "Shop & Service" venues are found within 500 m of location
0 "Indian Restaurant" venues are found within 500 m of location
4 "Metro Station" venues are found within 500 m of location
0 "Bus Stop" venues are found within 500 m of location
0 "Shop & Service" venues are found within 500 m of location
0 "Indian Restaurant" venues are found within 500 m of location
4 "Metro Station" venues are found within 500 m of location
0 "Bus Stop" venues are found within 500 m of location
0 "Shop & Service" venues are found within 500 m of location
0 "Indian Restaurant" venues are found within 500 m of location
0 "Metro Station" venues are found within 500 m of location
0 "Bus Stop" venues are found within 500 m of location
0 "Shop & Service" venues are found within 500 m of location
0 "Indian Restaurant" venues are found w

{'asset_id': 'e55f48ef-4c12-4ea3-9c14-a3d3b65d98d3',
 'bucket_name': 'myproject-donotdelete-pr-8jvbnevikkwojt',
 'file_name': 'venues12.csv',
 'message': 'File venues12.csv has been written successfully to the associated OS'}

In [946]:
#Lets read from previously explored and downloaded venue excel else API call limit is expiring
dfFinalVenues = pd.read_csv(project.get_file('venues12.csv'))

In [42]:
print('Total {} of venues are found'.format(len(dfFinalVenues)))

Total 284 of venues are found


In [43]:
dfFinalVenues.shape

(284, 7)

In [44]:
dfFinalVenues.head()

,Latitude,Longitude,Neighborhood,Venue,Venue Category,Venue Latitude,Venue Longitude
0,43.705369,-79.349372,Thorncliffe Park,Hamdi restaurant,Indian Restaurant,43.704601,-79.349699
1,43.705369,-79.349372,Thorncliffe Park,Trupti Indian Grocery,Indian Restaurant,43.707003,-79.350447
0,43.705369,-79.349372,Thorncliffe Park,TTC Bus #100 Flemingdon Park,Metro Station,43.707424,-79.348803
0,43.705369,-79.349372,Thorncliffe Park,Bus stop 8389,Bus Stop,43.702699,-79.346800
1,43.705369,-79.349372,Thorncliffe Park,TTC Stop #2979,Bus Stop,43.702430,-79.352345


In [45]:
#Check number of veneues in each category in each neighborhood
dfFinalVenues['Venue Category'].value_counts().to_frame(name='Count')

,Count
Bus Stop,122
Shop & Service,66
Metro Station,49
Indian Restaurant,47


In [46]:
dfFinalVenues.shape

(284, 7)

### Data analysis and visualization

In [47]:
dfFinalVenueCnt = []
dfFinalVenueCnt = dfFinalVenues.groupby(['Neighborhood','Venue Category'], as_index=False).count()
dfFinalVenueCnt.shape

(81, 7)

In [48]:
dfFinalVenueCnt.dtypes

Neighborhood       object
Venue Category     object
Latitude            int64
Longitude           int64
Venue               int64
Venue Latitude      int64
Venue Longitude     int64
dtype: object

In [49]:
dfFinalVenueCnt.drop(columns=['Latitude','Longitude','Venue Latitude', 'Venue Longitude'], inplace=True)

In [50]:
dfFinalVenueCnt.rename(columns={'Venue':'Count'}, inplace=True)
dfFinalVenueCnt.shape

(81, 3)

In [51]:
dfFinalVenueCnt = dfFinalVenueCnt.pivot(index='Neighborhood', columns='Venue Category', values='Count').fillna(0)

In [111]:
dfFinalVenueCnt.dtypes

Venue Category
Bus Stop             float64
Indian Restaurant    float64
Metro Station        float64
Shop & Service       float64
dtype: object

In [52]:
dfFinalVenueCnt

Venue Category,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
Neighborhood,,,,
Agincourt North,0.0,2.0,1.0,0.0
Alderwood,2.0,2.0,0.0,2.0
Bathurst Manor,2.0,2.0,1.0,2.0
Dorset Park,2.0,2.0,0.0,3.0
Flemingdon Park,1.0,2.0,1.0,2.0
Forest Hill North,2.0,2.0,0.0,0.0
Guildwood,4.0,1.0,1.0,0.0
Henry Farm,11.0,0.0,5.0,8.0
Highland Creek,1.0,0.0,0.0,0.0


In [110]:
#Data summary
dfFinalVenueCnt.describe()

Venue Category,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
count,30.000000,30.000000,30.000000,30.000000
mean,4.066667,1.566667,1.633333,2.200000
std,6.169633,2.192201,1.847334,3.854241
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,2.000000,1.000000,1.000000,1.000000
75%,4.750000,2.000000,2.750000,2.000000
max,26.000000,9.000000,7.000000,18.000000


In [112]:
#Finding correlations
dfFinalVenueCnt.corr()

Venue Category,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
Venue Category,,,,
Bus Stop,1.000000,0.782370,0.395534,0.870942
Indian Restaurant,0.782370,1.000000,-0.091676,0.802354
Metro Station,0.395534,-0.091676,1.000000,0.170475
Shop & Service,0.870942,0.802354,0.170475,1.000000


In [117]:
#p-values 
#Lets calculate p-value between Bus Stop, Indian Restaurant and Shop & Service

pearson_coef, p_value = stats.pearsonr(dfFinalVenueCnt['Bus Stop'], dfFinalVenueCnt['Indian Restaurant'])
print("The Pearson Correlation Coefficient b/w Bus Stop and Indian Restaurant is", pearson_coef, " with a P-value of P =", p_value)  

pearson_coef, p_value = stats.pearsonr(dfFinalVenueCnt['Bus Stop'], dfFinalVenueCnt['Shop & Service'])
print("The Pearson Correlation Coefficient b/w Bus Stop and Shop & Service is", pearson_coef, " with a P-value of P =", p_value)  

pearson_coef, p_value = stats.pearsonr(dfFinalVenueCnt['Shop & Service'], dfFinalVenueCnt['Indian Restaurant'])
print("The Pearson Correlation Coefficient b/w Shop & Service and Indian Restaurant", pearson_coef, " with a P-value of P =", p_value) 

The Pearson Correlation Coefficient b/w Bus Stop and Indian Restaurant is 0.782369569012  with a P-value of P = 3.26853819864e-07
The Pearson Correlation Coefficient b/w Bus Stop and Shop & Service is 0.870941872915  with a P-value of P = 3.88896687489e-10
The Pearson Correlation Coefficient b/w Shop & Service and Indian Restaurant 0.802354346792  with a P-value of P = 9.69636638855e-08


### Normalize using min max scaler

In [54]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [55]:
#dfFinalVenueCnt

In [85]:
VenueNorm = scaler.fit_transform(dfFinalVenueCnt)

In [87]:
dfVenueNorm = pd.DataFrame(VenueNorm, index=dfFinalVenueCnt.index,)

In [88]:
dfVenueNorm.columns=['Bus Stop', 'Indian Restaurant', 'Metro Station', 'Shop & Service']

In [90]:
dfVenueNorm.dtypes.index

Index(['Bus Stop', 'Indian Restaurant', 'Metro Station', 'Shop & Service'], dtype='object')

In [91]:
dfVenueNorm.reset_index(inplace=True)

In [92]:
dfVenueNorm["Neighborhood"][0]

'Agincourt North'

In [118]:
dfVenueNorm

,Neighborhood,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
0,Agincourt North,0.000000,0.222222,0.142857,0.000000
1,Alderwood,0.076923,0.222222,0.000000,0.111111
2,Bathurst Manor,0.076923,0.222222,0.142857,0.111111
3,Dorset Park,0.076923,0.222222,0.000000,0.166667
4,Flemingdon Park,0.038462,0.222222,0.142857,0.111111
5,Forest Hill North,0.076923,0.222222,0.000000,0.000000
6,Guildwood,0.153846,0.111111,0.142857,0.000000
7,Henry Farm,0.423077,0.000000,0.714286,0.444444
8,Highland Creek,0.038462,0.000000,0.000000,0.000000
9,Humber Summit,0.000000,0.111111,0.142857,0.000000


In [93]:
Priority = []

In [94]:
dfPriorityNeighborhood.set_index("Neighborhood")
for i, row in enumerate(dfVenueNorm.itertuples()):
    for key, row in enumerate (dfPriorityNeighborhood.itertuples()):
        if dfVenueNorm["Neighborhood"][i] == dfPriorityNeighborhood["Neighborhood"][key]:
           Priority.append({'Neighborhood':dfVenueNorm["Neighborhood"][i],
                              'Priority':dfPriorityNeighborhood["Priority"][key]})
dfPriority = pd.DataFrame(Priority, columns=['Neighborhood','Priority'])
dfPriorityNeighborhood.set_index("Neighborhood")
dfPriority

,Neighborhood,Priority
0,Agincourt North,4
1,Alderwood,3.5
2,Bathurst Manor,3.5
3,Dorset Park,4.5
4,Flemingdon Park,4.5
5,Forest Hill North,3.5
6,Guildwood,3.5
7,Henry Farm,4
8,Highland Creek,5
9,Humber Summit,4


In [95]:
venuePriorities = pd.merge(dfPriority, dfVenueNorm)

In [96]:
venuePriorities

,Neighborhood,Priority,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
0,Agincourt North,4,0.000000,0.222222,0.142857,0.000000
1,Alderwood,3.5,0.076923,0.222222,0.000000,0.111111
2,Bathurst Manor,3.5,0.076923,0.222222,0.142857,0.111111
3,Dorset Park,4.5,0.076923,0.222222,0.000000,0.166667
4,Flemingdon Park,4.5,0.038462,0.222222,0.142857,0.111111
5,Forest Hill North,3.5,0.076923,0.222222,0.000000,0.000000
6,Guildwood,3.5,0.153846,0.111111,0.142857,0.000000
7,Henry Farm,4,0.423077,0.000000,0.714286,0.444444
8,Highland Creek,5,0.038462,0.000000,0.000000,0.000000
9,Humber Summit,4,0.000000,0.111111,0.142857,0.000000


In [97]:
#Align it 
venuePriorities.drop(['Neighborhood','Priority'], axis=1, inplace=True)
venuePriorities

,Bus Stop,Indian Restaurant,Metro Station,Shop & Service
0,0.000000,0.222222,0.142857,0.000000
1,0.076923,0.222222,0.000000,0.111111
2,0.076923,0.222222,0.142857,0.111111
3,0.076923,0.222222,0.000000,0.166667
4,0.038462,0.222222,0.142857,0.111111
5,0.076923,0.222222,0.000000,0.000000
6,0.153846,0.111111,0.142857,0.000000
7,0.423077,0.000000,0.714286,0.444444
8,0.038462,0.000000,0.000000,0.000000
9,0.000000,0.111111,0.142857,0.000000


In [98]:
venuePriorities.shape

(30, 4)

In [99]:
dfPriority.shape

(30, 2)

In [100]:
venuePriorities.dtypes

Bus Stop             float64
Indian Restaurant    float64
Metro Station        float64
Shop & Service       float64
dtype: object

In [101]:
dfPriority = dfPriority.astype({"Priority": float})

In [102]:
dfPriority.dtypes

Neighborhood     object
Priority        float64
dtype: object

In [103]:
venueMatrix = venuePriorities.transpose().dot(dfPriority['Priority'])

In [104]:
venueMatrix

Bus Stop             18.365385
Indian Restaurant    19.944444
Metro Station        29.571429
Shop & Service       13.833333
dtype: float64

In [105]:
dfReco = (venuePriorities*venueMatrix).sum(axis=1)/venueMatrix.sum()
dfReco = dfReco.reset_index(name='Priority')

In [108]:
dfReco.max()

index       29.000000
Priority     0.751952
dtype: float64

### Thus we recommend Regent Park (max prority) neighborhood 

In [127]:
dfReco.sort_values(by='Priority', ascending=False)

,index,Priority
21,21,0.751952
12,12,0.715577
20,20,0.440446
7,7,0.428817
10,10,0.259419
11,11,0.259419
22,22,0.233726
14,14,0.206792
23,23,0.206792
27,27,0.172383
